# Amazon Bedrock Batch Inference

## 背景
- Bedrockのbatch推論のquotaがon demandと関係ない（だからquotaに引っ掛からなくてうれしい）と言う説
- text to image (SDXL or Titan or Both)で500枚くらいバッチ推論ジョブを発行してどんな風に実行が完了するか（もしくは完了しないか）を見てみる
- On demandのquotaより明らかに早かったら嬉しい


## 検証条件
- 推論方式: バッチ
- 生成: テキスト → 画像
- モデル:
    - amazon.titan-image-generator-v1
    - stability.stable-diffusion-xl-v1

## 結論

### 入出力データ
- インターフェイスはS3のみ。
- バッチ推論は入出力ファイルサイズ制限がある。モデル間での差分はない。
- https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch
- 最小も最大も定義されている。

#### 入力データフォーマット
- 一枚一枚に対してはオンデマンド推論と同じ。それをJSONLにしてS3に置く。
- リクエスト枚数が少ないと、ジョブ実行時の序盤にエラーで跳ね返される。
    - Titan: 4枚
    - SD: 10枚

#### 出力データフォーマット
それぞれのモデルで出力方法が全く異なるので注意

- Titan：単一のJSONLにバイナリで全画像が入力情報とセットで埋まっている。大量画像入っているのでデカJSONL
- SD：ファイル名に連番が入った一枚一枚のPNG

入力と出力の突き合わせ

- Titan：1行ずつの入力と出力がすぐ対応できるので、突き合わせしやすい
- SD：連番で突き合わせ作業するしかない

### ジョブ
- ジョブは並列実行されない。
- 複数のジョブを投げることは可能だが、順序保証されて（Undocumented）キューイングされて順次処理される。したがって前のジョブが完了するまで次のジョブは実行されない。
- よってジョブの並列実行による高速化はされない。
- またバッチ推論にすることでオンラインより高速化もされない。
- バッチ推論ジョブの速度はオンデマンド推論を同期処理しているような速度のため、非同期でオンデマンド処理させた方が高速化は可能。

### 結果
600枚全て成功

- Titan: 12 sec / image
- SD: 7 sec /image　← 70分くらい

条件全て揃えられているわけではないので注意

## 公開情報
- 開発者ドキュメント: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference.html
- Quota: https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch
- コードサンプル: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-example.html

## 環境準備

### 2024.2.1
- Public Preview
- 利用方法
    - REST API: あると思うが面倒
    - CLI: 無さそう
    - SDK: プレビューのがある
        - PythonとJavaのみ
        - ただし他サービスと同様、プレビュー状態のAPIは一般公開SDKに含まれていない
        - そのため以下の公式whlからインストールが必要
        - https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip
    - コンソール: 無さそう

Python SDKを使うこととし、以下でSDKのインストールをする

In [ ]:
! chmod +x install_sdk.sh
! ./install_sdk.sh

上記実行後、カーネルの再起動が必要

In [ ]:
from IPython import Application
Application.instance().kernel.do_shutdown(True)

```aws s3 cp``` などをするため、AWS CLIがあるか確認

In [ ]:
! aws --version

## 入力データフォーマット

参考: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-data.html

以下、サンプルの入力JSON

入力JSON Linesフォーマット
```JSON
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {JSON body}
}
...
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {JSON body}
}
```

Titan imageの場合の推論入力JSON: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html#model-parameters-titan-image-api


```JSON
{
    "inputText": string,
    "textGenerationConfig": {
        "temperature": float,  
        "topP": float,
        "maxTokenCount": int,
        "stopSequences": [string]
    }
}
```

Tiatanの場合のバッチ推論入力JSON Lines
```JSON
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {
        "inputText": string,
        "textGenerationConfig": {
            "temperature": float,  
            "topP": float,
            "maxTokenCount": int,
            "stopSequences": [string]
        }
    }
}
...
{
    "recordId": "12 character alphanumeric string",
    "modelInput": {
        "inputText": string,
        "textGenerationConfig": {
            "temperature": float,  
            "topP": float,
            "maxTokenCount": int,
            "stopSequences": [string]
        }
    }
}
```

## 共通条件設定

In [1]:
image_size:int = 1024
cfg_scale:int = 8.0
seed:int = 0

In [2]:
height:int = image_size
width:int = image_size

image_generation_config:dict = {
    "numberOfImages": 1,
    "quality": "standard",
    "height": height,
    "width": width,
    "cfgScale": cfg_scale,
    "seed": seed
}

In [3]:
def return_body_for_claude(prompt:str) -> dict:
    return {
        "prompt": f"\n\nHuman:{prompt}\n\nAssistant:",
        # "temperature": float,
        # "top_p": float,
        # "top_k": int,
        # "max_tokens_to_sample": int,
        # "stop_sequences": [string]
    }

def return_body_for_titan_image(prompt:str) -> dict:
    return {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": image_generation_config
    }

def return_body_for_sd(prompt:str) -> dict:
    return {
        "text_prompts": [{"text": prompt}],
        "height": height,
        "width": width,
        "cfg_scale": cfg_scale,
        # "clip_guidance_preset": string,
        # "sampler": string,
        # "samples",
        "seed": seed,
        # "steps": int,
        # "style_preset": string,
        # "extras": JSON object
    }

functions = {
    'anthropic.claude-v2:1': return_body_for_claude,
    'amazon.titan-image-generator-v1': return_body_for_titan_image,
    'stability.stable-diffusion-xl-v1': return_body_for_sd,
}

## 検証

### 条件設定

バッチ推論は入出力ファイルサイズ制限がある。モデル間での差分はない。
https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch

最小も最大も定義されている。そのためリクエスト枚数が少ないと、ジョブ実行時の序盤にエラーで跳ね返される。

- Titan: 4枚
- SD: 10枚

程度が最小。

### ジョブ作成

In [1]:
import bedrock
_bedrock:bedrock.Bedrock = bedrock.Bedrock(region="us-east-1")

from batch import Batch

In [5]:
number_of_prompts:int = 10

batch = Batch(
    model_id = "amazon.titan-image-generator-v1",
    functions = functions,
    bedrock = _bedrock,
)
input_key, output_dir = batch.create_inputs_by(
    prompts = (f"{i+2} dogs running at a park." for i in range(number_of_prompts))
)
! aws s3 cp $input_key -

{"recordId": "000000000000", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "2 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "width": 1024, "cfgScale": 8.0, "seed": 0}}}
{"recordId": "000000000001", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "3 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "width": 1024, "cfgScale": 8.0, "seed": 0}}}
{"recordId": "000000000002", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "4 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "width": 1024, "cfgScale": 8.0, "seed": 0}}}
{"recordId": "000000000003", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "5 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "

In [8]:
batch.create_job(input_key, output_dir)

{'ResponseMetadata': {'RequestId': '935acd12-a6d5-4611-a17e-45e2b559dcf9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 09 Feb 2024 08:34:15 GMT',
   'content-type': 'application/json',
   'content-length': '85',
   'connection': 'keep-alive',
   'x-amzn-requestid': '935acd12-a6d5-4611-a17e-45e2b559dcf9'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-east-1:624045005200:model-invocation-job/hnkwljadyzsn'}

In [13]:
_bedrock.group_jobs_by_status()

status
Completed    69
Failed       28
Submitted     3
dtype: int64

In [3]:
number_of_prompts:int = 10

In [6]:
ti = Batch(
    model_id = "amazon.titan-image-generator-v1",
    functions = functions,
    bedrock = _bedrock,
)
ti_input_key, ti_output_dir = ti.create_inputs_by(
    prompts = (f"{i+2} dogs running at a park." for i in range(number_of_prompts))
)
! aws s3 cp $ti_input_key -

{"recordId": "000000000000", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "2 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "width": 1024, "cfgScale": 8.0, "seed": 0}}}
{"recordId": "000000000001", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "3 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "width": 1024, "cfgScale": 8.0, "seed": 0}}}
{"recordId": "000000000002", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "4 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "width": 1024, "cfgScale": 8.0, "seed": 0}}}
{"recordId": "000000000003", "modelInput": {"taskType": "TEXT_IMAGE", "textToImageParams": {"text": "5 dogs running at a park."}, "imageGenerationConfig": {"numberOfImages": 1, "quality": "standard", "height": 1024, "

In [7]:
sd = Batch(
    model_id = "stability.stable-diffusion-xl-v1",
    functions = functions,
    bedrock = _bedrock,
)
sd_input_key, sd_output_dir = sd.create_inputs_by(
    prompts = (f"{i+2} dogs running at a park." for i in range(number_of_prompts))
)
! aws s3 cp $sd_input_key -

{"recordId": "000000000000", "modelInput": {"text_prompts": [{"text": "2 dogs running at a park."}], "height": 1024, "width": 1024, "cfg_scale": 8.0, "seed": 0}}
{"recordId": "000000000001", "modelInput": {"text_prompts": [{"text": "3 dogs running at a park."}], "height": 1024, "width": 1024, "cfg_scale": 8.0, "seed": 0}}
{"recordId": "000000000002", "modelInput": {"text_prompts": [{"text": "4 dogs running at a park."}], "height": 1024, "width": 1024, "cfg_scale": 8.0, "seed": 0}}
{"recordId": "000000000003", "modelInput": {"text_prompts": [{"text": "5 dogs running at a park."}], "height": 1024, "width": 1024, "cfg_scale": 8.0, "seed": 0}}
{"recordId": "000000000004", "modelInput": {"text_prompts": [{"text": "6 dogs running at a park."}], "height": 1024, "width": 1024, "cfg_scale": 8.0, "seed": 0}}
{"recordId": "000000000005", "modelInput": {"text_prompts": [{"text": "7 dogs running at a park."}], "height": 1024, "width": 1024, "cfg_scale": 8.0, "seed": 0}}
{"recordId": "000000000006",

In [ ]:
ti.create_job(ti_input_key, ti_output_dir)

In [ ]:
sd.create_job(sd_input_key, sd_output_dir)

In [ ]:
_bedrock.group_jobs_by_status()

### 最大キュー数の確認

In [ ]:
from time import sleep
while True:
    ti.create_job(ti_input_key, ti_output_dir)
    sd.create_job(sd_input_key, sd_output_dir)
    print(_bedrock.group_jobs_by_status())
    sleep(1)

## テキスト

In [4]:
claude = Batch(
    model_id = 'anthropic.claude-v2:1',
    functions = functions,
    bedrock = _bedrock.client,
)
claude_input_key, claude_output_dir = claude.create_inputs_by(
    prompts = (f"{i} + 1 = ?" for i in range(number_of_prompts))
)
! aws s3 cp $claude_input_key -

{"recordId": "000000000000", "modelInput": {"prompt": "\n\nHuman:0 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000001", "modelInput": {"prompt": "\n\nHuman:1 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000002", "modelInput": {"prompt": "\n\nHuman:2 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000003", "modelInput": {"prompt": "\n\nHuman:3 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000004", "modelInput": {"prompt": "\n\nHuman:4 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000005", "modelInput": {"prompt": "\n\nHuman:5 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000006", "modelInput": {"prompt": "\n\nHuman:6 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000007", "modelInput": {"prompt": "\n\nHuman:7 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000008", "modelInput": {"prompt": "\n\nHuman:8 + 1 = ?\n\nAssistant:"}}
{"recordId": "000000000009", "modelInput": {"prompt": "\n\nHuman:9 + 1 = ?\n\nAssistant:"}}


In [5]:
claude.create_job(claude_input_key, claude_output_dir)

{'ResponseMetadata': {'RequestId': '92e529d2-d765-4299-8513-df30a2530ba5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 09 Feb 2024 08:14:50 GMT',
   'content-type': 'application/json',
   'content-length': '85',
   'connection': 'keep-alive',
   'x-amzn-requestid': '92e529d2-d765-4299-8513-df30a2530ba5'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-east-1:624045005200:model-invocation-job/b91z3s38a1zj'}

In [13]:
_bedrock.group_jobs_by_status()

status
Completed     69
Failed        28
InProgress     1
Stopped        2
dtype: int64

In [14]:
_bedrock.get_dataframe_of_jobs(max_results=3)

,jobName,modelId,roleArn,status,submitTime,lastModifiedTime
jobArn,,,,,,
arn:aws:bedrock:us-east-1:624045005200:model-invocation-job/hnkwljadyzsn,amazon.titan-image-generator-v1-10-20240209-17...,arn:aws:bedrock:us-east-1::foundation-model/am...,arn:aws:iam::624045005200:role/Admin_role_for_...,InProgress,2024-02-09 08:34:15.152000+00:00,2024-02-09 08:34:55.311000+00:00
arn:aws:bedrock:us-east-1:624045005200:model-invocation-job/b91z3s38a1zj,anthropic.claude-v2-1-10-20240209-171450,arn:aws:bedrock:us-east-1::foundation-model/an...,arn:aws:iam::624045005200:role/Admin_role_for_...,Stopped,2024-02-09 08:14:50.629000+00:00,2024-02-09 08:37:52.516000+00:00
arn:aws:bedrock:us-east-1:624045005200:model-invocation-job/uveo7osa8mzf,anthropic.claude-v2-1-10-20240209-165126,arn:aws:bedrock:us-east-1::foundation-model/an...,arn:aws:iam::624045005200:role/Admin_role_for_...,Stopped,2024-02-09 07:51:26.726000+00:00,2024-02-09 08:36:52.658000+00:00


In [14]:
claude.stop()

AttributeError: 'Batch' object has no attribute 'stop'

In [22]:
claude.wait()
! aws s3 cp $claude_output_dir$claude.id/manifest.json.out -

ID: uveo7osa8mzf
Name: anthropic.claude-v2-1-10-20240209-165126
Submitted


ジョブの状態はこちら: https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-list.html

ジョブは並列実行されない。

複数のジョブを投げることは可能だが、ジョブが順序保証されて（Undocumented）キューイングされて順次処理される。したがって前のジョブが完了するまで次のジョブは実行されない。

よってジョブの並列実行による高速化はされない。

またバッチ推論にすることでオンラインより高速化もされない。

バッチ推論ジョブの速度はオンデマンド推論を同期処理しているような速度のため、非同期でオンデマンド処理させた方が高速化は可能。

### 画像表示
それぞれのモデルで出力方法が全く異なるので注意

- Titan：単一のJSONLにbase64で全画像が入力情報とセットで埋まっている
- SD：ファイル名に連番が入った一枚一枚のPNG

入力と出力の突き合わせ

- Titan：1行ずつの入力と出力がすぐ対応できるので、突き合わせしやすい
- SD：連番で突き合わせ作業するしかない

In [ ]:
# import base64
# from io import BytesIO
# from PIL import Image

# output_dir:str = f"{dir}/{job_id}"

# def get_content_binary(key:str):
#     output_obj = bucket.Object(key=output_key).get()
#     binary_contents = output_obj.get("Body").read()
#     return binary_contents


# def display_image(image_bytes):
#     image = Image.open(BytesIO(image_bytes))
#     image.show()

In [ ]:
# import json

# if "titan" in model_id:
#     output_key = f"{output_dir}/{model_id}.jsonl.out"
#     binary_contents = get_content_binary(output_key)

#     for line in BytesIO(binary_contents):
#         content = json.loads(line.decode("utf-8"))
#         # finish_reason = content.get("error")
#         # if finish_reason is not None: print(f"Image generation error. Error is {finish_reason}")
#         print(content.get("modelInput").get("textToImageParams").get("text"))
#         base64_image = content.get("modelOutput").get("images")[0]
#         base64_bytes = base64_image.encode('ascii')
#         image_bytes = base64.b64decode(base64_bytes)
#         display_image(image_bytes)
# elif "stable" in model_id:
#     for record_id in range(number_of_prompts):
#         output_key = f"{output_dir}/{model_id}.{str(record_id).zfill(12)}.1.png"
#         binary_contents = get_content_binary(output_key)
#         print(output_key)
#         display_image(binary_contents)

## 結論

### 入出力データ
- インターフェイスはS3のみ。
- バッチ推論は入出力ファイルサイズ制限がある。モデル間での差分はない。
- https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html#quotas-batch
- 最小も最大も定義されている。

#### 入力データフォーマット
- 一枚一枚に対してはオンデマンド推論と同じ。それをJSONLにしてS3に置く。
- リクエスト枚数が少ないと、ジョブ実行時の序盤にエラーで跳ね返される。
    - Titan: 4枚
    - SD: 10枚

#### 出力データフォーマット
それぞれのモデルで出力方法が全く異なるので注意

- Titan：単一のJSONLにバイナリで全画像が入力情報とセットで埋まっている。大量画像入っているのでデカJSONL
- SD：ファイル名に連番が入った一枚一枚のPNG

入力と出力の突き合わせ

- Titan：1行ずつの入力と出力がすぐ対応できるので、突き合わせしやすい
- SD：連番で突き合わせ作業するしかない

### ジョブ
- ジョブは並列実行されない。
- 複数のジョブを投げることは可能だが、順序保証されて（Undocumented）キューイングされて順次処理される。したがって前のジョブが完了するまで次のジョブは実行されない。
- よってジョブの並列実行による高速化はされない。
- またバッチ推論にすることでオンラインより高速化もされない。
- バッチ推論ジョブの速度はオンデマンド推論を同期処理しているような速度のため、非同期でオンデマンド処理させた方が高速化は可能。

### 結果
600枚全て成功

- Titan: 12 sec / image
- SD: 7 sec /image　← 70分くらい

条件全て揃えられているわけではないので注意

## 単発推論
まずは単発を確認

### Titan image
https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-titan-image.html

In [ ]:
prompt:str = "A dog running at a park."

from func import generate_image_by

In [ ]:
body = {
    "taskType": "TEXT_IMAGE",
    "textToImageParams": {
        "text": prompt
    },
    "imageGenerationConfig": image_generation_config
}
generate_image_by(model_id="amazon.titan-image-generator-v1", body=body)

### Stable Diffusion
https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion-1-0-text-image.html

In [ ]:
body = {
    "text_prompts": [{"text": prompt}],
    "height": height,
    "width": width,
    "cfg_scale": cfg_scale,
    # "clip_guidance_preset": string,
    # "sampler": string,
    # "samples",
    "seed": seed,
    # "steps": int,
    # "style_preset": string,
    # "extras" :JSON object
}
generate_image_by(model_id="stability.stable-diffusion-xl-v1", body=body)